# Introdução:
Temos um conjunto de dados de vendas de veículos usados e, em posse disso, vamos:

1 - Fazer um levantamento de quais estatísticas nos são mais relevantes;

2 - Fazer uma análise exploratória de dados e criar e responder 3 perguntas de negócio, e também responder as seguintes perguntas:

   2.a - Qual o melhor estado na base de dados para se vender um veículo e por que? 
   
   2.b - Qual o melhor estado para se comprar uma Picape com transmissão automática e por que?
   
   2.c - Qual o melhor estado para se comprar carros que estejam dentro da garantia de fábrica e por que?
    
3 - como será feita a previsão do preço e por que? qual tipo de problema estamos resolvendo? qual modelo melhor se aproxima dos dados e quais seus prós e contras? Qual medida de performance do modelo foi escolhida e por que?

## Bibliotecas:

Abaixo, importamos todas as bibliotecas que serão usadas ao longo deste projeto.

In [ ]:
import pandas            as pd
import numpy             as np
import seaborn           as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

sns.set()

## Funções:

In [ ]:
def double_bar_plot(series1,
                    series2,
                    figsize=(12,6),
                    xlabel1=None,
                    xlabel2=None,
                    ylabel1=None,
                    ylabel2=None,
                    title1=None,
                    title2=None,
                   color1='mediumaquamarine',
                   color2='sandybrown'):
    plt.figure(figsize=figsize, facecolor='moccasin')

    plt.subplot(1,2,1)
    sns.barplot(x=series1.index, y=series1, color=color1, alpha=0.7);
    plt.xlabel(xlabel1)
    plt.ylabel(ylabel1)
    plt.title(title1)
    plt.xticks(rotation=64)

    plt.subplot(1, 2, 2)
    sns.barplot(x=series2.index, y=series2, color=color2, alpha=0.7);
    plt.xlabel(xlabel2)
    plt.ylabel(ylabel2)
    plt.title(title2)
    plt.xticks(rotation=64)

    plt.show()

## Dados:
Temos um conjunto de treino e um conjunto de teste já previamente separados. Aqui, carregamos ambos. Os dados foram recebidos em arquivos .csv que estavam codificados em UTF-16LE, e as colunas estavam separadas pelo caracter de espaçamento `\t`. 

Além de carregar os dados, vamos tentar extrair algumas informações preliminares: quais features estão envolvidas, qual o tipo de cada feature e algumas estatísticas básicas de features numéricas.

In [ ]:
treino = pd.read_csv("cars_train.csv", delimiter='\t', encoding='utf-16-le')
teste  = pd.read_csv("cars_test.csv",  delimiter='\t', encoding='utf-16-le')

# quantidade de dados e features com os quais estamos trabalhando:
print("treino:", treino.shape)
print("teste:", teste.shape)
print()

# Todas as features com as quais estamos lidando: 
print("features: ")
treino.info()

In [ ]:
treino.describe()

Pudemos ver acima pelas contagens que faltam valores de algumas features. Em outras palavras, há dados nulos em nosso conjunto de dados. Vamos procurar entender melhor estes dados nulos para decidirmos se vamos tratá-los, preenchê-los ou mesmo descartá-los:

In [ ]:
# dados nulos em teste: 
dados_nulos = (teste.isnull().sum()[teste.isnull().sum() > 0])
print("dados nulos em teste:")
print(dados_nulos)

print()

# dados nulos em treino: 
dados_nulos = (treino.isnull().sum()[treino.isnull().sum() > 0])
print("dados nulos em treino: ")
print(dados_nulos)

Vamos investigar isso melhor. Abaixo, algumas coisas chamam a atenção. 

A coluna `'veiculo_alienado'` possui apenas dados nulos. Isso pode querer dizer que nenhum veículo da base de dados é alieanado, mas, de todo modo, o dado é uma constante e portanto trivial. Descartaremos esta coluna.

A coluna `'num_fotos'` possui relativamente poucos dados nulos. Substituíremos esses dados nulos pela parte inteira da média de fotos por veículo considerando todo o conjunto de dados(treino e teste). Parece fazer mais sentido substituir por um valor médio do que por 0 pois parece improvável que se anuncie um veículo à venda na internet sem que haja fotos do veículo.

Como se pode ver abaixo, todas as outras possuem apenas dois valores. Não há como distinguir se há dados ausentes nessas colunas ou não pois, ao que parece, os dados ausentes se confundiriam com dados negativos. Por exemplo, não temos como saber se um veículo não está licenciado de fato ou se a informação apenas está ausente. Optamos, em um primeiro momento, por supor que não há dados ausentes nestas colunas e subsituímos seus valores por `True` ou `False`. 

In [ ]:
for coluna in dados_nulos.index:
    print("{}(teste):\n".format(coluna), teste[coluna].unique())
    print("{}(treino):\n".format(coluna), treino[coluna].unique())
    print()

In [ ]:
# media de numero de fotos por anúncio
(teste['num_fotos'].sum()+treino['num_fotos'].sum())/(treino.shape[0]+teste.shape[0])

In [ ]:
# descarta a coluna 'veiculo_alienado':
treino.drop(columns='veiculo_alienado', inplace=True)
teste.drop(columns='veiculo_alienado', inplace=True)

# transforma os dados das demais colunas com dados nulos,
# com exceção de 'num_fotos' e 'veiculo_alienado':
for coluna in dados_nulos.index[1:-1]:
    treino[coluna] = treino[coluna].apply(lambda x: (type(x) == str))
    teste[coluna]  =  teste[coluna].apply(lambda x: (type(x) == str))    
    
# substitui dados nulos em fotos por um valor próximo à média: 
treino['num_fotos'].fillna(10, inplace=True)
teste['num_fotos'].fillna(10, inplace=True)

Agora, como podemos ver, já não há mais dados nulos nos conjuntos de teste e treino.

In [ ]:
# dados nulos em teste: 
dados_nulos = (teste.isnull().sum()[teste.isnull().sum() > 0])
print("dados nulos em teste:")
print(dados_nulos)

print()

# dados nulos em treino: 
dados_nulos = (treino.isnull().sum()[treino.isnull().sum() > 0])
print("dados nulos em treino: ")
print(dados_nulos)

Agora, vamos renomear algumas colunas para facilitar a manipulação dos dados. 

In [ ]:
print("antes:\n", treino.columns)

nomes = {'num_fotos'           : 'fotos', 
         'ano_de_fabricacao'   : 'fabricacao',
         'num_portas'          : 'portas',
         'entrega_delivery'    : 'delivery',
         'dono_aceita_troca'   : 'aceita_troca',
         'veiculo_único_dono'  : 'unico_dono',
         'garantia_de_fábrica' : 'garantia',
         'veiculo_licenciado'  : 'licenciado',
         'estado_vendedor'     : 'uf'
        }
teste.rename(columns=nomes, inplace=True)
treino.rename(columns=nomes, inplace=True)
print()
print("depois:\n", treino.columns)

Vamos também dar uma forma mais conveniente à coluna com dados dos estados dos vendedores. Podemos observar que não há, dentre os dados de treino, vendedores no Distrito Federal(DF) e no Amapá(AP), e dentre os dados de teste, além deste já citados, não há vendedores do Maranhão (MA).  

In [ ]:
treino['uf'] = treino['uf'].apply(lambda x: x[-3:-1])
teste['uf'] = teste['uf'].apply(lambda x: x[-3:-1])

print("quantidade de estados (treino):", len(treino['uf'].unique()))
print(treino['uf'].unique())
print()
print("quantidade de estados (teste):", len(teste['uf'].unique()))
print(teste['uf'].unique())

Abaixo, podemos ver que São Paulo é o estado onde mais tem veículos anunciados. Isso não deveria ser surpreendente, porque São Paulo é o estado com maior poder econômico do Brasil, segunda maior renda per capita, e também a maior população. Roraima, por outro lado, é o estado com menor poder econômico e menor população, o que pode justificar sua baixa quantidade de anúncios nestes dados. 

Entretanto, esses dados não necessariamente refletem o cenário nacional como um todo, pois não sabemos a origem dos dados. Por exemplo, os dados podem ter vindo de uma plataforma que investe mais em marketing no estado de São Paulo do que nos outros. O Distrito Federal tem a maior renda per capita e a maior densidade demográfica do País, contando com mais de quatro vezes a população de Roraima, mas sequer consta no conjunto de dados com o qual trabalhamos. 

Dito isso, não podemos usar apenas esses dados para falar de compra e vendas a um nível verdadeiramente nacional. 

Abaixo, podemos ver que há muito mais anúncios em SP do que nos demais estados:


In [ ]:
x1 = treino['uf'].value_counts() + teste['uf'].value_counts()
x1.sort_values(ascending=False, inplace=True)

plt.figure(figsize = (16,6), facecolor='moccasin')

plt.subplot(1, 2, 1)
plt.title('Total de Anúncios por União Federativa')
sns.barplot(x=x1.index, y=x1.values, alpha=0.7, label='treino')
plt.xlabel('UF')
plt.ylabel('Total de Anúncios')
plt.xticks(rotation=64)

sp = treino[treino['uf'] == 'SP']['uf'].value_counts().sum() + teste[teste['uf'] == 'SP']['uf'].value_counts().sum()    
demais = treino[treino['uf'] != 'SP']['uf'].value_counts().sum() + teste[teste['uf'] != 'SP']['uf'].value_counts().sum()
plt.subplot(1, 2, 2)
plt.bar(['SP', "Demais UF's"], [sp, demais], color =['cyan', 'khaki'],width = 0.4)

plt.show()

# Melhor estado para se vender carros de marcas populares
Primeiro, precisamos definir o que são marcas populares. A expressão "carro popular" costuma se referir a "carros de entrada", carros mais baratos do que média. Vamos primeiro olhar para a média de preços dos carros anunciados: 

In [ ]:
preco_medio = treino['preco'].mean()
print("preço médio:",preco_medio)

Vemos que a média de preços dos veículos anunciados está por volta de 133 mil reais. Vamos agora olhar para a média dos preços dos veículos por marca:

In [ ]:
medias_marcas = (treino.groupby('marca')['preco'].mean().sort_values())
medias_marcas

Vamos considerar 'marcas populares' como sendo marcas cuja média dos preços está abaixo do preço médio calculado acima: 

In [ ]:
marcas_populares = medias_marcas.loc[lambda x: x<preco_medio].index
marcas_populares = treino[treino['marca'].isin(marcas_populares)]

In [ ]:
populares = marcas_populares['uf'].value_counts()

plt.figure(figsize = (16,4), facecolor='moccasin')

plt.subplot(1, 2, 1)
plt.bar(populares.index,populares, color='mediumaquamarine');
plt.title("Carros 'populares' por UF")
plt.xticks(rotation=64)
plt.show()

Vamos agora olhar para a média de preços de marcas populares por UF: 

In [ ]:
populares_medias_uf = marcas_populares.groupby('uf')['preco'].mean().sort_values(ascending=False)

plt.figure(figsize = (16,6), facecolor='moccasin')
sns.barplot(x=populares_medias_uf.index, y=populares_medias_uf, color='mediumaquamarine', alpha=0.7, label='treino');
plt.ylabel('Preço médio de carros de marcas populares')
plt.title('Preço médio de carros de marcas populares por UF(quanto maior, melhor para se vender)')
plt.xticks(rotation=64)
plt.show()

Aqui já nos parece que Piauí é a melhor opção para se vender carros de marcas populares(conforme nossa definição de marcas populares), pois é o estado onde a média dos preços de carros dessas marcas é maior. 

Agora, a fim de ver a variabilidade dos preços dos veículos das marcas populares por UF, vamos olhar para o desvio padrão dos preços de carros de marcas populares nesses estados que tem a média mais alta:

In [ ]:
populares_desvios_uf = marcas_populares.groupby('uf')['preco'].std().sort_values(ascending=False)

plt.figure(figsize = (16,6), facecolor='moccasin')
sns.barplot(x=populares_desvios_uf.index, y=populares_desvios_uf, color='mediumaquamarine', alpha=0.7, label='treino');
plt.ylabel('Desvio Padrão de carros de marcas populares')
plt.title('Desvio Padrão de carros de marcas populares por UF(quanto menor, melhor para se vender)')
plt.xticks(rotation=64)
plt.show()

Uma média alta nos diz que é bom vender veículos em um estado porque pode ser mais fácil de se competir no mercado local. Um desvio padrão baixo nos diz que o preço dos carros varia pouco. Se tivermos uma média alta e um desvio padrão baixo, significa que temos preços altos e poucas opções de preços. Vemos que Maranhão e Rondônia tem preço médio de veículos de marcas populares bastante altos, embora não tanto quanto Piauí, mas em compensação tem Desvio Padrão bem mais baixo. 

Para decidir, agora vamos olhar para o Coeficiente de Variação, que é a razão entre o Desvio Padrão e a Média. Nós procuramos aqui o Coeficiente de Variação mais baixo pois, além de nos dizer qual estado apresenta menor variabilidade de dados em relação à média (o que seria bom para vendas), por ser a razão entre o Desvio Padrão e a Média, pode ser, neste caso, encarado como uma relação de "custo/benefício" entre essas duas medidas, pois queremos maximizar a média, o que contribui para um menor Coeficiente de Variação, e minimizar o desvio padrão, que também contribui para um menor Coeficiente de Variação:  

In [ ]:
d = marcas_populares.groupby('uf')['preco'].std()
m = marcas_populares.groupby('uf')['preco'].mean()

populares_CV_uf = (d/m).sort_values()
populares_CV_uf

plt.figure(figsize = (16,6), facecolor='moccasin')
sns.barplot(x=populares_CV_uf.index, y=populares_CV_uf, color='mediumaquamarine', alpha=0.7, label='treino');
plt.ylabel('Coeficiente de Variação')
plt.title('Coeficiente de Variação de carros de marcas populares por UF(quanto menor, melhor para se vender)')
plt.xticks(rotation=64)
plt.show()

Curiosamente, Roraima é o estado com menor coeficiente de variação, muito embora tenha a menor média de preços, o que não seria bom sinal. Esta análise puramente numérica com base apenas nos dados que nos foram fornecidos é um indício muito forte de Roraima é o melhor estado para se vender veículos e, se eu for obrigado a dizer qual o melhor lugar para se vender carros de marcas populares, conforme definimos acima, eu digo que o melhor estado é Roraima. 

Entretanto, olhando para a quantidade de veículos de marcas populares anunciados em cada um dos estados, parece difícil decidir entre Maranhão, Piauí, Roraima e Rondonia, pois temos muito poucos dados para querer tomar decisões baseadas apenas em estatística, e em todos esses estados há poucos veículos há vendas do que nós definimos acima como 'marcas populares'. Talvez com um conjunto de dados maior ou uma lista pré-definida de 'marcas populares', pudéssemos ter melhores insights sobre qual o melhor estado.    

In [ ]:
marcas_populares['uf'].value_counts()

# Picapes com Transmissão Automática
Abaixo, podemos ver que há mais picapes com câmbio automático anunciadas em SP do que em todos as outras UF's combinadas. Isto é um forte indicativo de que SP é o melhor estado na nossa base de dados para comprar uma picape com transmissão automática simplesmente porque lá há mais opções. Mas vamos investigar isso um pouco melhor. 

In [ ]:
picape_automatica = treino[(treino['tipo'] == 'Picape') & (treino['cambio'] == 'Automática')]
x1 = picape_automatica['uf'].value_counts()

sp = picape_automatica[picape_automatica['uf'] == 'SP'].shape[0]
demais = picape_automatica[picape_automatica['uf'] != 'SP'].shape[0]
data = pd.Series({'SP':sp, 'Outros Estados':demais})

double_bar_plot(x1,
               data,
               figsize = (16,6),
               ylabel1='Total de Anúncios',
               ylabel2="No. de Anúncios",
               title1='Total de picapes com câmbio automático anunciadas por União Federativa',
               title2="Anúncios de Picapes Automáticas: SP vs. Demais UF's",
               color1='mediumaquamarine',
               color2='sandybrown')

Podemos ver (abaixo) que o preço médio de Picapes com transmissão automática de São Paulo não é dos mais altos, mas o desvio padrão é. Quanto mais alto é o desvio padrão, mais os dados estão dispersos. Isso significa que, dos dados que temos em mãos, não só há mais opções em São Paulo do que em todo o resto do país, como também a dispersão dos preços em torno da média só não é maior que a de Santa Catarina, que por sinal possui uma menor média de preço. Quanto maior o desvio padrão, mais temos opções de preços. 

In [ ]:
d = picape_automatica.groupby('uf')['preco'].std().sort_values(ascending=False)
m = picape_automatica.groupby('uf')['preco'].mean().sort_values()

double_bar_plot(m,d,
                title1="média por UF (quanto menor, melhor)",
                title2="desvio padrão por UF (quanto maior, melhor)")

Como podemos ver acima, Santa Catarina parece um bom estado para se comprar Picapes Automáticas pois tem uma média de preço menor que a de São Paulo, um Desvio Padrão Maior, e consequentemente, um Coeficiente de Variação Maior. 

Plotamos abaixo a distribuição dos das picapes com transmissão automática no estado de São Paulo e Santa Catarina. Vemos que realmente há muitas opções nas mais diversas faixas de preço. 

In [ ]:
plt.figure(figsize = (12,6), facecolor='moccasin')
# Filtramos veículos acima de R$ 500.000,00 para melhorar a visualização.
# Há veículos com preços acima disso, mas não chegariam a ser visíveis neste gráfico  
plt.hist(picape_automatica[(picape_automatica['uf'] == 'SP')]['preco'], bins=156, color='mediumaquamarine');
plt.xlabel("Preço (R$)")
plt.ylabel("Quantidade de Anúncios")
plt.title("Veículos Anunciados em São Paulo")
plt.show()

plt.figure(figsize = (12,6), facecolor='moccasin')
# Filtramos veículos acima de R$ 500.000,00 para melhorar a visualização.
# Há veículos com preços acima disso, mas não chegariam a ser visíveis neste gráfico  
plt.hist(picape_automatica[(picape_automatica['uf'] == 'SC')]['preco'], bins=156, color='mediumaquamarine');
plt.xlabel("Preço (R$)");
plt.ylabel("Quantidade de Anúncios")
plt.title("Veículos Anunciados em Santa Catarina");
plt.show()

Vamos agora olhar para o Coeficiente de Variação, dessa vez procurando o maior:

In [ ]:
d = picape_automatica.groupby('uf')['preco'].std()
m = picape_automatica.groupby('uf')['preco'].mean()

picape_automatica_CV_uf = (d/m).sort_values(ascending=False)

plt.figure(figsize = (16,6), facecolor='moccasin')
sns.barplot(x=picape_automatica_CV_uf.index, y=picape_automatica_CV_uf, color='mediumaquamarine', alpha=0.7, label='treino');
plt.ylabel('Coeficiente de Variação')
plt.title('Coeficiente de Variação de preços de Picapes Automáticas por UF(quanto maior, melhor para compra)')
plt.xticks(rotation=64)
plt.show()

Santa Catarina tem um Coeficiente de Variação maior, o que já era esperado, dado que a média de preços em SC é meor que em SP e que o desvio padrão é maior em SC. Mas isso não é conclusivo, primeiro porque há muito mais opções em São Paulo do que em Santa Catarina, e segundo porque temos a disposição mais dados para analisar. Vamos olhar para fatores qualitativos agora: 

In [ ]:
delivery = picape_automatica.groupby('uf')['delivery'].sum()
delivery_pct = picape_automatica.groupby('uf')['delivery'].sum()/picape_automatica['uf'].value_counts()
delivery.sort_values(inplace=True)
delivery_pct.sort_values(inplace=True)

double_bar_plot(series1=delivery,
               series2=delivery_pct,
               ylabel1='Quantidade de Anúncios',
               ylabel2='Porcentagem',
               title1='Anúncios cujos vendedores fazem entrega do veículo',
               title2='Porcentagem de Anúncios que fazem entrega'
               )               

In [ ]:
aceita_troca = picape_automatica.groupby('uf')['aceita_troca'].sum()
aceita_troca_pct = picape_automatica.groupby('uf')['aceita_troca'].sum()/picape_automatica['uf'].value_counts()

aceita_troca.sort_values(inplace=True)
aceita_troca_pct.sort_values(inplace=True)

double_bar_plot(aceita_troca,
                    aceita_troca_pct,
                    ylabel1='Quantidade de Anúncios',
                    ylabel2='Porcentagem',
                    title1='Anúncios cujos vendedores aceitam troca',
                    title2='Porcentagem de Anúncios que aceitam troca',
                   color1='mediumaquamarine',
                   color2='sandybrown')

In [ ]:
unico_dono = picape_automatica.groupby('uf')['unico_dono'].sum()
unico_dono_pct = picape_automatica.groupby('uf')['unico_dono'].sum()/picape_automatica['uf'].value_counts()

unico_dono.sort_values(inplace=True)
unico_dono_pct.sort_values(inplace=True)

double_bar_plot(unico_dono,
                    unico_dono_pct,
                    ylabel1='Quantidade de Anúncios',
                    ylabel2='Porcentagem',
                    title1='Veículos que possuíram único dono',
                    title2='Porcentagem de veículos que possuíram único dono')

Em um caso real, seguiríamos explorando as demais características do veículo. Aqui, por questões de tempo e também para poupar o leitor, concluíremos que São Paulo é o melhor estado para se comprar Picapes Automáticas porque há uma média baixa de preços, há muito mais anúncios do que em todo o resto do país combinados, há mais opções que fazem entrega do veículo, há mais anúncios de veículos que possuíram único dono, há mais anúncios cujo vendedor aceita troca... 
Talvez um último fator seja ainda relevante de ser checado: a quantidade de marcas disponíveis para a compra. Abaixo, vemos que São Paulo também oferece mais opções de marcas para se comprar. 

In [ ]:
marcas = picape_automatica.groupby('uf')['marca'].nunique()
marcas.sort_values(inplace=True)

plt.figure(figsize=(16,6), facecolor='moccasin')
sns.barplot(x=marcas.index, y=marcas, alpha=0.7);
plt.ylabel("Total de marcas diferentes")
plt.title("Marcas de veículos disponíveis por estado")
plt.xticks(rotation=64)
plt.show()

# Carros que ainda estão dentro da garantia de fábrica
A análise aqui é análoga à análise feita anteriormente, por isso seremos um pouco mais diretos aqui. Também cabe observar que a discrepância da quantidade de anúncios de SP para o resto do país em nossa base de dados é tão grande que o problema é quase trivial. 

In [ ]:
garantia = treino[(treino['garantia'] == True)]
x1 = garantia['uf'].value_counts()

sp = garantia[garantia['uf'] == 'SP'].shape[0]
demais = garantia[garantia['uf'] != 'SP'].shape[0]
data = pd.Series({'SP':sp, 'Outros Estados':demais})

double_bar_plot(x1,
               data,
               figsize = (16,6),
               ylabel1='Total de Anúncios',
               ylabel2="No. de Anúncios",
               title1='Total de Veículos dentro da Garantia de Fábrica por União Federativa',
               title2="SP vs. Demais UF's",
               color1='mediumaquamarine',
               color2='sandybrown')

d = garantia.groupby('uf')['preco'].std().sort_values(ascending=False)
m = garantia.groupby('uf')['preco'].mean().sort_values()

double_bar_plot(m,d,
                title1="média por UF (quanto menor, melhor)",
                title2="desvio padrão por UF (quanto maior, melhor)")

marcas = garantia.groupby('uf')['marca'].nunique()
marcas.sort_values(inplace=True)

plt.figure(figsize=(16,6), facecolor='moccasin')
sns.barplot(x=marcas.index, y=marcas, alpha=0.7);
plt.ylabel("Total de marcas diferentes")
plt.title("Marcas de veículos ainda na garantia disponíveis por estado")
plt.xticks(rotation=64)
plt.show()


aceita_troca = garantia.groupby('uf')['aceita_troca'].sum()
aceita_troca_pct = garantia.groupby('uf')['aceita_troca'].sum()/garantia['uf'].value_counts()

aceita_troca.sort_values(inplace=True)
aceita_troca_pct.sort_values(inplace=True)

double_bar_plot(aceita_troca,
                    aceita_troca_pct,
                    ylabel1='Quantidade de Anúncios',
                    ylabel2='Porcentagem',
                    title1='Anúncios de veículos em garantia cujos vendedores aceitam troca',
                    title2='Porcentagem de Anúncios que aceitam troca',
                   color1='mediumaquamarine',
                   color2='sandybrown')


unico_dono = garantia.groupby('uf')['unico_dono'].sum()
unico_dono_pct = garantia.groupby('uf')['unico_dono'].sum()/garantia['uf'].value_counts()

unico_dono.sort_values(inplace=True)
unico_dono_pct.sort_values(inplace=True)

double_bar_plot(unico_dono,
                    unico_dono_pct,
                    ylabel1='Quantidade de Anúncios',
                    ylabel2='Porcentagem',
                    title1='Veículos dentro de garantia que possuíram único dono',
                    title2='Porcentagem de veículos que possuíram único dono')

Os gráficos acima sugerem que São Paulo também é o melhor lugar para se comprar veículos ainda em garantia de fábrica por haver maior disponibilidade de marcas, mais veículos ainda em garantia que possuíram único dono, cujo dono aceita troca, etc. 

# Regressão
Temos aqui um problema de regressão, e não de classificação, já que queremos prever uma variável (preço) que pode assumir, a princípio, qualquer valor positivo. Faremos aqui uma regressão linear. Começamos preparando o conjunto que temos para treino: 

In [ ]:
X = pd.get_dummies(treino.drop('preco', axis=1))
y = treino['preco']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Agora, criamos e treinamos o modelo de regressão linear: 

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

Por fim, fazemos as previsões e avaliamos a performance do modelo:

In [ ]:
y_pred = model.predict(X_train)

# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)